# Hidden Number Problem

O problema do número escondido está relacionado com duas funções sobre elementos do corpo $Z_p$ e números naturais:
1. A primeira função, a **norma de um elemento em $Z_p$**, é definida da seguinte maneira:
    $\mid\mid X\mid\mid_p=$min$(x, p-x)$
1. A segunda função devolve os **$k$ bits mais significativos do seu argumento**, um número natural, módulo $p$: $msb_{k,p}(x)$

## Boneh & Venkatesan Algorithm

In [662]:
import numpy as np
import random as rand
import sage.crypto.lattice as lat
from sage.modules.free_module_integer import IntegerLattice

class BV:
    def __init__(self, p, n, k, s):
        self.p = ZZ(p)
        self.n = n
        if k>= sqrt(len(self.p.bits())):
            self.lambd = 2 ** (k+1)
            self.x = [ZZ.random_element(self.p) for i in range(0, self.n)]
            self.define_target(s)
            self.gen_lattice()

    def gen_lattice(self):
        ''' Calculate the lattice base
        '''
        b_1 = self.lambd * self.p * identity_matrix(ZZ, self.n)
        b_2 = self.lambd * matrix(self.x)
        self.G = block_matrix([[b_1, 0],[b_2, 1]])
        self.lat = IntegerLattice(self.G)
        self.lat_reduced = np.array(self.lat.reduced_basis)
    
    def msb(self, x):
        ''' Return an approximation of the k most significative bits of x
        '''
        valid_u = False
        while (not valid_u):
            u = ZZ.random_element(self.p)
            xu = (x - u) % self.p
            norm_xu = min(xu, self.p - xu)
            valid_u = (norm_xu <= self.p//self.lambd)
        return u

    def define_target(self, s):
        u = [self.msb(s * x_i) for x_i in self.x]
        self.targ = [self.lambd * u_i for u_i in u]
        self.targ.append(0)
        
    def solve_exact(self):
        cv_exact = self.lat.closest_vector(tuple(self.targ))
        return cv_exact
        
    def solve_approx(self):
        l_b = matrix(self.lat.reduced_basis)
        t_b = matrix(1, self.n+1, [-t_i for t_i in self.targ])
        z_b = matrix(self.n+1, 1, [0]*(self.n+1))
        M_b = matrix(1, 1, [self.lambd * self.p ** 2])
        l1 = block_matrix(2, 2, [ [l_b, z_b], [t_b, M_b]])
        l1_reduced = IntegerLattice(l1).reduced_basis
        delta = np.array(l1_reduced[self.n+1][:-1])
        y1 = matrix(list(self.targ + delta))
        return y1    

In [683]:
p = 7
n = 2
k = 3
s = 4
bv = BV(p, n, k, s)
exact = bv.solve_exact()
approx = bv.solve_approx()
print("x: " + str(bv.x))
print("target: " + str(bv.targ))
print("")
print("L: ")
print(bv.G)
print("")
print("Exact CVP: " + str(exact))
print("Approx CVP: " + str(approx))

x: [6, 6]
target: [48, 48, 0]

L: 
[112   0|  0]
[  0 112|  0]
[-------+---]
[ 96  96|  1]

Exact CVP: (48, 48, -3)
Approx CVP: [48 48 -3]
